# Revised case normalization for Winterthur 2020

Notes:



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/Winterthur.xlsx'),
         'KSW',
         '2020',
         'Änderungen_Winterthur_2020')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Winterthur.xlsx', hospital_name_db='KSW', year='2020', sheet='Änderungen_Winterthur_2020')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-14 08:50:25.733 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 214 cases for KSW 2020
2022-11-14 08:50:25.744 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:63 - TYPES:
tranche                   string
datum                     string
fid                       string
case_id                   object
patient_id                object
kkik                      string
gender                    string
age_years                  int64
bfs_code                  string
duration_of_stay           int64
pflegetage neu            string
pccl                       int64
pccl neu                  string
old_pd                    string
primary_diagnosis         string
added_icds                string
removed_icds              string
added_chops               string
removed_chops             string
drg                       string
drg neu                   string
cw alt                    string
cw neu         

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,20049284027,E66EFC8932E6E4BF,M,66,25,20049284027,C241,C241,M200,3,H01B,[K830],[],[],[]
1,20134514027,89F52CDD41B3C501,M,73,5,20134514027,I210,I210,M990,3,F24E,[S2240],[],[],[0040::20200402]
2,20071661027,C183570C2D12CCAF,M,61,10,20071661027,C9000,C9000,M100,3,R61D,[I483],[],[],[]
3,20350424027,E93C58A92B1946D2,W,77,10,20350424027,M167,M167,M200,3,I46D,[D689],[],[],[]
4,20359514027,DE96E9E3453EADE4,W,80,6,20359514027,C220,C220,M100,3,H61B,"[K703, L8914]",[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,20814284027,9272923D724DAED5,W,85,7,20814284027,I1190,I1100,M990,3,F67B,[I1320],[],[],[]
209,20156534027,6D2E237F3F63E2F8,W,66,18,20156534027,I5001,I5001,M100,3,F62C,"[L8927, I361]",[],[],[]
210,20466614027,B6ED36FAD74E3867,M,77,12,20466614027,N10,N10,M200,2,L18B,"[R64, C772, C775]",[],[],[]
211,22257944027,B4C9627D1558EB1F,W,73,4,22257944027,R502,N390,M990,2,T62C,[B962],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

2022-11-14 08:50:27.467 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 13056 rows from the DB, for the hospital 'KSW' in 2020
2022-11-14 08:50:27.483 | WARNING  | src.revised_case_normalization.notebook_functions.revise:revise:47 - 3 rows could not be matched, given ['age_years', 'case_id_norm', 'duration_of_stay', 'gender']


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,389189,21320891027,L270,"[I1090, I481, Z921, G406, I8720, ...]",8611::20200129,[],M,81,0,0,6,0,01,2020-01-29,0,00,2020-02-04
1,389250,20071661027,C9000,"[N1793, I481, Z921, E785, I1090, ...]",992551::20200130,"[4131::20200129, 921906::20200129, 889717::20200127, 9961::20200202]",M,61,0,0,10,0,01,2020-01-26,0,00,2020-02-05
2,389272,22769804027,I214,"[I5014, I2519, J9601, U990, J9611, ...]",99B712::20200517,[998425::20200517],M,77,0,0,4,16,01,2020-05-17,0,07,2020-05-21
3,389290,20542931027,K255,"[B980, K650, S3616, Y849]",4441::20200116,"[434221::20200116, 5425::20200116, 542110::20200116, 99B811::20200116]",M,42,0,0,7,0,01,2020-01-16,0,00,2020-01-23
4,389494,21347201027,O2688,"[O096, Z370]",741X10::20191230,[741X20::20191230],W,32,0,0,4,0,01,2019-12-30,0,00,2020-01-03


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops,aimedic_id,case_id_db,patient_id_db,age_days,admission_weight,gestation_age,admission_date,grouper_admission_type,discharge_date,grouper_discharge_type,ventilation_hours
28,21343271027,0F8B21DB5B81266F,W,47,24,21343271027,C031,C031,M200,3,D37A,[T8651],[R0200],[],"[311::20200107, 4311::20200122]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,21576701027,898A8ACB29A00D43,M,50,17,21576701027,I81,I81,M100,3,H12B,[S3611],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,22443764027,0,W,35,8,22443764027,O611,O611,M300,3,O01F,[],[],[741X20::20200617],[741X10::20200617],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-14 08:50:28.164 | INFO     | src.revised_case_normalization.notebook_functions.group:group:83 - Grouping 170 cases ...
2022-11-14 08:50:29.939 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:88 - Grouped 170 cases into: 170 revisions, 1760 diagnoses rows, 655 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,389189,J61D,0.72,0.72,3,2022-12-31
1,389250,R61C,1.74,1.74,4,2022-12-31
2,389272,F60A,1.36,1.36,4,2022-12-31
3,389290,G19A,2.57,2.57,4,2022-12-31
4,389494,O01F,1.01,1.01,0,2022-12-31
...,...,...,...,...,...,...
165,401689,G04Z,3.18,3.18,4,2022-12-31
166,401699,H09A,3.23,3.23,4,2022-12-31
167,401715,F59F,0.97,1.49,3,2022-12-31
168,401837,G48C,0.95,0.95,3,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-14 08:50:29.956 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:249 - Trying to insert 170 cases into the 'Revisions' table ...
2022-11-14 08:50:30.699 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:275 - Deleted 170 rows from the "Revisions" table, which is about to be updated
2022-11-14 08:50:30.865 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:286 - Inserted 170 cases into the 'Revisions' table
2022-11-14 08:50:30.870 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:298 - Trying to insert 1760 rows into the 'Diagnoses' table ...
2022-11-14 08:50:34.203 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:336 - Inserted 1760 rows into the 'Diagnoses' table
2022-11-14 08:50:34.206 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:347 - Trying to insert 655 rows into the 'Procedure